In [2]:
from google.colab import files
uploaded = files.upload()


Saving speech.txt.csv to speech.txt (1).csv


In [3]:
import pandas as pd
import os

fname = next((f for f in os.listdir() if f.endswith('.csv')), None)
if not fname:
    raise FileNotFoundError("No CSV found. Upload speech.txt.csv in the Files pane or re-run upload cell.")
print("Using file:", fname)

df = pd.read_csv(fname, header=0)
display(df.head())


passages = df.astype(str).apply(lambda r: " ".join(r.values), axis=1).tolist()
print(f"\nLoaded {len(passages)} passages. Example:\n")
print(passages[0][:400])


Using file: speech.txt.csv


,The real remedy is to destroy the belief in the sanctity of the shastras. How do you expect to
0,succeed if you allow the shastras to continue ...
1,a stand against the scriptures. Either you mus...
2,believing in the shastras. You cannot have bot...
3,reform. It is a problem of overthrowing the au...
4,"the sanctity of the shastras, they will never ..."



Loaded 7 passages. Example:

succeed if you allow the shastras to continue to be held as sacred and infallible? You must take


In [4]:

!pip install -q scikit-learn

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel


vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
tfidf_matrix = vectorizer.fit_transform(passages)
print("TF-IDF matrix shape:", tfidf_matrix.shape)


TF-IDF matrix shape: (7, 79)


In [5]:
def answer_question_tfidf(question, top_k=3):
    """
    Returns top_k passages most similar to the question using cosine similarity on TF-IDF.
    """
    q_vec = vectorizer.transform([question])

    cos_sim = linear_kernel(q_vec, tfidf_matrix).flatten()

    top_indices = cos_sim.argsort()[::-1][:top_k]
    results = [(i, cos_sim[i], passages[i]) for i in top_indices]
    return results


q = input("Type your question (example: What does the speech say about caste?):\n")
if q.strip():
    res = answer_question_tfidf(q, top_k=3)
    print("\nTop passages:\n")
    for rank, (idx, score, text) in enumerate(res, start=1):
        print(f"--- Passage {rank} (row {idx}) — score {score:.3f} ---\n")
        print(text[:800])
        print("\n")
else:
    print("No question entered.")


Type your question (example: What does the speech say about caste?):
What does the speech say about caste?

Top passages:

--- Passage 1 (row 2) — score 0.222 ---

believing in the shastras. You cannot have both. The problem of caste is not a problem of social


--- Passage 2 (row 1) — score 0.201 ---

a stand against the scriptures. Either you must stop the practice of caste or you must stop


--- Passage 3 (row 4) — score 0.198 ---

the sanctity of the shastras, they will never be able to get rid of caste. The work of social reform


